## Load

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append('..')
from gesetz import Gesetz

In [3]:
import pickle
with open("pickles/laws_links_topics.pickle", "rb") as fp:   # Unpickling
    Gesetz.collected_laws = pickle.load(fp)

## Graph

In [4]:
import networkx as nx

In [5]:
ovw = nx.MultiDiGraph()

interesting_topics = ["GesR","ArbR"]

for law in Gesetz.collected_laws.values():
    for link in law.links:
        if link.target ==None:
            continue

        targ_top = Gesetz.collected_laws[link.target].get_topic()
        if targ_top not in interesting_topics:
            targ_top = 0
        src_top = law.get_topic()
        if src_top not in interesting_topics:
            src_top = 0
        ovw.add_edge(src_top,targ_top)

In [6]:
print(ovw.nodes)

[0, 'GesR', 'ArbR']


## start visualization

In [7]:
values = {}
for (s,t,n) in ovw.edges:
    if (s,t) not in values or values[(s,t)]<n:
        values[(s,t)]=n

In [8]:
len(values)

9

In [9]:
source = []
target = []
value = []

for (s,t) in values:
    if (s != 0) and (t==s or t==0):
        source.append(s)
        target.append(t)
        value.append(values[(s,t)])



nodes = [node for node in ovw.nodes]

source_scaled = source
target_scaled = target
label = []
for index in range(len(nodes)):
    node = nodes[index]
    source_scaled = [index if src==node else src for src in source_scaled]
    target_scaled = [index+(len(nodes)) if tgt==node else tgt for tgt in target_scaled]

In [10]:
source_scaled

[1, 1, 2, 2]

In [11]:
target_scaled

[4, 3, 5, 3]

In [12]:
label = [""+str(lbl).zfill(2)+"_out" if lbl != 0 else "others" for lbl in ovw.nodes]
for nde in ovw.nodes:
    label.append(""+str(nde).zfill(2)+"_in"if nde != 0 else "others")
print(label)

['others', 'GesR_out', 'ArbR_out', 'others', 'GesR_in', 'ArbR_in']


In [13]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label,
      color = ["gray","goldenrod","green","gray","goldenrod","green"]
    ),
    link = dict(
      source = source_scaled, # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = target_scaled,
      value = value,
      color = ["goldenrod","goldenrod","green","green"]
  ))])

fig.update_layout(font_size=10,font=dict(family='Times New Roman',size=12))
fig.show()

# FHM

In [14]:
from graph import Graph
Graph.set_big(Gesetz.collected_laws)

{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


In [15]:
topics = set([law.get_topic() for law in Gesetz.collected_laws.values()])
FHMs = []
for topic in topics:
    print(topic)
    graph = Graph.build_topics([topic])
    if len(graph.G_rel.edges) >0:
        FHMs.append((graph.get_flow(),topic))

FHMs.sort()

SachenR
not found: []
StatistikR
not found: []
BankR
not found: []
StrafR
not found: []
OrganR
not found: []
VersorgungsR
not found: []
PatentR
not found: []
KommR
not found: []
ArbR
not found: []
AgrarR
not found: []
VerkehrsR
not found: []
MedR
not found: []
InsolvenzR
not found: []
EntsorgungsR
not found: []
GebührenR
not found: []
BerufsR
not found: []
ProduktR
not found: []
UmweltR
not found: []
WohnR
not found: []
ProzessR
not found: []
GesR
not found: []
FinMarktR
not found: []
AsylR
not found: []
BeamtenR
not found: []
SteuerR
not found: []
KriegsfolgenR
not found: []
BauR
not found: []
BahnR
not found: []
WahlR
not found: []


In [16]:
print(FHMs)

[(0.0, 'FinMarktR'), (0.0928616550852811, 'SteuerR'), (0.09859154929577463, 'BauR'), (0.24293785310734461, 'ProzessR'), (0.2571428571428571, 'PatentR'), (0.2735949098621421, 'StrafR'), (0.2845303867403315, 'BeamtenR'), (0.3224043715846995, 'BahnR'), (0.3294573643410853, 'UmweltR'), (0.3648474430597336, 'GesR'), (0.3669724770642202, 'MedR'), (0.435820895522388, 'SachenR'), (0.4444444444444444, 'KommR'), (0.4931506849315068, 'StatistikR'), (0.5128205128205128, 'ArbR'), (0.6, 'VerkehrsR'), (0.7280219780219781, 'VersorgungsR'), (0.8467153284671532, 'OrganR'), (0.92, 'BerufsR'), (1.0, 'BankR'), (1.0, 'EntsorgungsR'), (1.0, 'GebührenR'), (1.0, 'InsolvenzR'), (1.0, 'KriegsfolgenR'), (1.0, 'ProduktR'), (1.0, 'WahlR')]


In [17]:
file = open("FHM.txt","w",encoding="utf8")


for r,n in FHMs:
    file.write(str(r).ljust(19,"0")+": ")
    file.write(n+"\n")

file.close()
